In [2]:
import pyprob
import itertools
from pyprob import Model, InferenceEngine
from pyprob.distributions import Uniform, Normal, Categorical, Poisson
from pyprob.util import to_variable, to_numpy

import torch
import math
import numpy as np
import scipy
import scipy.linalg as la
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
colors = [cm.inferno(x) for x in np.linspace(0, 1, 5)]
%matplotlib inline

ImportError: cannot import name 'to_variable'

In [ ]:
# This will have to be modfied accordingly
min_energy_deposit = 0.05  # caloutils::minEnergyDeposit in sharpa_tau_decay.cpp

def particle_name(pid):
    if abs(pid) > pid:
        return 'anti-' + pypdt.get(abs(pid)).name
    else:
        return pypdt.get(pid).name

def plot_distribution(dist, plot_samples=2000, title='', ground_truth_trace=None, file_name=None, infer_observation_importance=1.):
    if dist.length > 0:
        #[ground_truth_trace.result[3], ground_truth_trace.result[0], ground_truth_trace.result[1], ground_truth_trace.result[2]]
        dist_resampled = dist.resample(plot_samples)
        dist_px = dist_resampled.map(lambda x: float(x[0]))
        dist_py = dist_resampled.map(lambda x: float(x[1]))
        dist_pz = dist_resampled.map(lambda x: float(x[2]))
        dist_channel = dist_resampled.map(lambda x: int(x[3]))
        dist_final_state_particles = dist_resampled.map(lambda x: x[4:4+(30*8)].view(-1,8))
        dist_final_state_particles_px = dist_final_state_particles.map(lambda x: x[:, 0])
        dist_final_state_particles_py = dist_final_state_particles.map(lambda x: x[:, 1])
        dist_final_state_particles_pz = dist_final_state_particles.map(lambda x: x[:, 2])
        dist_final_state_particles_E = dist_final_state_particles.map(lambda x: x[:, 3])
        dist_final_state_particles_theta = dist_final_state_particles.map(lambda x: x[:, 4])
        dist_final_state_particles_phi = dist_final_state_particles.map(lambda x: x[:, 5])
        dist_final_state_particles_id = dist_final_state_particles.map(lambda x: x[:, 6])
        dist_final_state_particles_calo_visible = dist_final_state_particles.map(lambda x: x[:, 7])

        dist_px_samples = dist_px.values
        dist_py_samples = dist_py.values
        dist_pz_samples = dist_pz.values
        dist_channel_samples = dist_channel.values
        dist_final_state_particles_px_samples = dist_final_state_particles_px.values
        dist_final_state_particles_py_samples = dist_final_state_particles_py.values
        dist_final_state_particles_pz_samples = dist_final_state_particles_pz.values
        dist_final_state_particles_E_samples = dist_final_state_particles_E.values
        dist_final_state_particles_theta_samples = dist_final_state_particles_theta.values
        dist_final_state_particles_phi_samples = dist_final_state_particles_phi.values
        dist_final_state_particles_id_samples = dist_final_state_particles_id.values
        dist_final_state_particles_calo_visible_samples = dist_final_state_particles_calo_visible.values

        final_state_particles_pid_flattened = []
        final_state_particles_calo_visible_flattened = []
        final_state_particles_px_flattened_samples = []
        final_state_particles_py_flattened_samples = []
        final_state_particles_pz_flattened_samples = []
        final_state_particles_E_flattened_samples = []
        final_state_particles_theta_flattened_samples = []
        final_state_particles_phi_flattened_samples = []
        for i in range(dist_resampled.length):
            for j in range(30):
                pid = dist_final_state_particles_id_samples[i][j]
                if float(pid) > -99999:
                    final_state_particles_pid_flattened.append(pid)
                    final_state_particles_calo_visible_flattened.append(float(dist_final_state_particles_calo_visible_samples[i][j]))
                    final_state_particles_px_flattened_samples.append(float(dist_final_state_particles_px_samples[i][j]))
                    final_state_particles_py_flattened_samples.append(float(dist_final_state_particles_py_samples[i][j]))
                    final_state_particles_pz_flattened_samples.append(float(dist_final_state_particles_pz_samples[i][j]))
                    final_state_particles_E_flattened_samples.append(float(dist_final_state_particles_E_samples[i][j]))
                    final_state_particles_theta_flattened_samples.append(float(dist_final_state_particles_theta_samples[i][j]))
                    final_state_particles_phi_flattened_samples.append(float(dist_final_state_particles_phi_samples[i][j]))

        dist_channel_flattened = pyprob.distributions.Empirical(dist_channel_samples, combine_duplicates=True)
        dist_channel_flattened_values = dist_channel_flattened.values
        dist_channel_flattened_weights = [float(w) for w in dist_channel_flattened.weights]
        dist_channel_str = 'Channel probabilities: ' + ', '.join(['{}: {:.3f}'.format(v, w) for v, w in zip(dist_channel_flattened_values, dist_channel_flattened_weights)])

        dist_final_state_particles_id_flattened = pyprob.distributions.Empirical(final_state_particles_pid_flattened, combine_duplicates=True).map(lambda x: particle_name(int(x)))
        dist_final_state_particles_id_flattened_values = dist_final_state_particles_id_flattened.values
        dist_final_state_particles_id_flattened_weights = [float(w) for w in dist_final_state_particles_id_flattened.weights]
        dist_final_state_particles_id_flattened_str = 'Final state particle probabilities: ' + ', '.join(['{}: {:.3f}'.format(v, w) for v, w in zip(dist_final_state_particles_id_flattened_values, dist_final_state_particles_id_flattened_weights)])

        dist_final_state_particles_calo_visible_flattened = pyprob.distributions.Empirical(final_state_particles_calo_visible_flattened, combine_duplicates=True).map(lambda x: 'Yes' if int(x) == 1 else 'No')
        dist_final_state_particles_calo_visible_flattened_values = dist_final_state_particles_calo_visible_flattened.values
        dist_final_state_particles_calo_visible_flattened_weights = [float(w) for w in dist_final_state_particles_calo_visible_flattened.weights]

        fig = plt.figure(figsize=(30, 15))
        gs1 = gridspec.GridSpec(3, 5, width_ratios=[2,1,1,1,1], wspace=0.2, hspace=0.4)
        gs2 = gridspec.GridSpec(3, 5, width_ratios=[2.4,1,1,1,1], wspace=0.2, hspace=0.4)
        ax1 = fig.add_subplot(gs1[0], projection='3d')
        ax2 = fig.add_subplot(gs2[1])
        ax3 = fig.add_subplot(gs2[2])
        ax4 = fig.add_subplot(gs2[3])
        ax5 = fig.add_subplot(gs2[4])
        ax6 = fig.add_subplot(gs1[5], projection='3d')
        ax7 = fig.add_subplot(gs2[6])
        ax8 = fig.add_subplot(gs2[7])
        ax9 = fig.add_subplot(gs2[8])
        ax10 = fig.add_subplot(gs2[9])
        ax11 = fig.add_subplot(gs1[10], projection='3d')
        ax12 = fig.add_subplot(gs2[11])
        ax13 = fig.add_subplot(gs2[12])
        ax14 = fig.add_subplot(gs2[13])
        ax15 = fig.add_subplot(gs2[14])
        ax2.text(0, 1.5, title)
        ax2.text(0, 1.4, dist_channel_str)
        ax2.text(0, 1.3, dist_final_state_particles_id_flattened_str)

        ax2.title.set_text('Channel')
        # ax2.text(0.5, 0.5, 'mean={:.3f} (solid), stddev={:.3f}'.format(float(dist_channel.mean), float(dist_channel.stddev)), ha='center', va='center', transform=ax2.transAxes)
        ax3.title.set_text('p_x')
        ax3.text(0.5, 0.5, 'mean={:.3f} (solid), stddev={:.3f}'.format(float(dist_px.mean), float(dist_px.stddev)), ha='center', va='center', transform=ax3.transAxes)
        ax4.title.set_text('p_y')
        ax4.text(0.5, 0.5, 'mean={:.3f} (solid), stddev={:.3f}'.format(float(dist_py.mean), float(dist_py.stddev)), ha='center', va='center', transform=ax4.transAxes)
        ax5.title.set_text('p_z')
        ax5.text(0.5, 0.5, 'mean={:.3f} (solid), stddev={:.3f}'.format(float(dist_pz.mean), float(dist_pz.stddev)), ha='center', va='center', transform=ax5.transAxes)

        _ = ax2.hist(dist_channel_samples, bins=np.arange(-0.5,38+0.5,1), density=True, alpha=0.6, color=colors_inferno[0])
        ax2.set_xlim([-1,39])
        ax2.set_ylim([0,1])
        # ax2.axvline(float(dist_channel.mean), color='gray', linestyle='solid', linewidth=1)
        ax2.xaxis.set_minor_locator(MultipleLocator(1))
        _ = ax3.hist(dist_px_samples, bins=np.arange(-2.5,2.5+0.1,0.1), density=True, alpha=0.6, color=colors_inferno[1])
        ax3.set_xlim([-2.5,2.5])
        ax3.set_ylim([0,10])
        ax3.axvline(float(dist_px.mean), color='gray', linestyle='solid', linewidth=1)
        _ = ax4.hist(dist_py_samples, bins=np.arange(-2.5,2.5+0.1,0.1), density=True, alpha=0.6, color=colors_inferno[2])
        ax4.set_xlim([-2.5,2.5])
        ax4.set_ylim([0,10])
        ax4.axvline(float(dist_py.mean), color='gray', linestyle='solid', linewidth=1)
        _ = ax5.hist(dist_pz_samples, bins=np.arange(43,47+0.08,0.08), density=True, alpha=0.6, color=colors_inferno[3])
        ax5.set_xlim([43,47])
        ax5.set_ylim([0,12.5])
        ax5.axvline(float(dist_pz.mean), color='gray', linestyle='solid', linewidth=1)

        dist_calo = dist.resample(int(plot_samples/4)).map(lambda x: x[4+(30*8):])
        calo_simulated = dist_calo.mean.view(35,35,20).data.numpy()
        calo_simulated_flat = calo_simulated.reshape(-1)
        ax6.title.set_text('Simulated calorimeter mean')
        ax6.set_xlabel('x')
        ax6.set_ylabel('y')
        ax6.set_zlabel('z')
        ax6.set_aspect('equal')
        cmbr = mpl.colors.LinearSegmentedColormap.from_list('blue_to_red',['b','r'])
        cnorm = mpl.colors.Normalize(vmin=calo_simulated_flat.min(),vmax=calo_simulated_flat.max())
        cpick = mpl.cm.ScalarMappable(norm=cnorm,cmap=cmbr)
        cpick.set_array([])
        if calo_simulated_flat.max() - calo_simulated_flat.min() > 0:
            ix,iy,iz = np.mgrid[-3:3:35j,-3:3:35j,4:15:20j]
            calo_simulated_flat_normalized = np.clip((calo_simulated_flat - calo_simulated_flat.min())/(calo_simulated_flat.max() - calo_simulated_flat.min()), 0, 1)
            colors = [[x,0,1-x,x] for x in calo_simulated_flat_normalized]
            ax6.scatter(ix, iy, iz, s=100, c=colors, marker='o', edgecolor='none')
            plt.colorbar(cpick, ax=ax6, fraction=0.02)

        # dist_final_state_particles_all_px = []
        # dist_final_state_particles_all_py = []
        # dist_final_state_particles_all_pz = []
        # for i in range(dist_final_state_particles.length):
        #     px = dist_final_state_particles_px_samples[i]
        #     py = dist_final_state_particles_py_samples[i]
        #     pz = dist_final_state_particles_pz_samples[i]
        #     dist_final_state_particles_all_px.append(px[px>-99999].data.cpu().numpy())
        #     dist_final_state_particles_all_py.append(py[py>-99999].data.cpu().numpy())
        #     dist_final_state_particles_all_pz.append(pz[pz>-99999].data.cpu().numpy())
        _ = ax8.hist(final_state_particles_px_flattened_samples, bins=40, histtype='bar', density=True, alpha=0.6, color=colors_inferno[1])
        ax8.title.set_text('Final state particles p_x')
        _ = ax9.hist(final_state_particles_py_flattened_samples, bins=40, histtype='bar', density=True, alpha=0.6, color=colors_inferno[2])
        ax9.title.set_text('Final state particles p_y')
        _ = ax10.hist(final_state_particles_pz_flattened_samples, bins=40, histtype='bar', density=True, alpha=0.6, color=colors_inferno[3])
        ax10.title.set_text('Final state particles p_z')

        ax7.title.set_text('Final state particles')
        ax7.bar(range(len(dist_final_state_particles_id_flattened_weights)), dist_final_state_particles_id_flattened_weights, align='center', alpha=0.6, color=colors_inferno[0])
        # for tick in ax7.get_xticklabels():
            # tick.set_rotation(45)
        # xticks = ax7.get_xticklabels()
        ax7.set_xticks(range(len(dist_final_state_particles_id_flattened_values)))
        ax7.set_xticklabels(dist_final_state_particles_id_flattened_values, rotation=45, ha='right')

        ax12.title.set_text('Final state particles calorimeter visible')
        ax12.bar(range(len(dist_final_state_particles_calo_visible_flattened_weights)), dist_final_state_particles_calo_visible_flattened_weights, tick_label=dist_final_state_particles_calo_visible_flattened_values, align='center', alpha=0.6, color=colors_inferno[0])

        ax13.title.set_text('Final state particles E')
        ax13.hist(final_state_particles_E_flattened_samples, bins=40, histtype='bar', density=True, alpha=0.6, color=colors_inferno[0])

        ax14.title.set_text('Final state particles theta')
        ax14.hist(final_state_particles_theta_flattened_samples, bins=40, histtype='bar', density=True, alpha=0.6, color=colors_inferno[0])

        ax15.title.set_text('Final state particles phi')
        ax15.hist(final_state_particles_phi_flattened_samples, bins=40, histtype='bar', density=True, alpha=0.6, color=colors_inferno[0])


        if ground_truth_trace is not None:
            calo_observed = ground_truth_trace.samples_observed[0].distribution.mean.view(35,35,20).data.numpy()
            calo_observed_flat = calo_observed.reshape(-1)
            ax1.title.set_text('Observed calorimeter')
            ax1.set_xlabel('x')
            ax1.set_ylabel('y')
            ax1.set_zlabel('z')
            ax1.set_aspect('equal')
            cmbr = mpl.colors.LinearSegmentedColormap.from_list('blue_to_red',['b','r'])
            cnorm = mpl.colors.Normalize(vmin=calo_observed_flat.min(),vmax=calo_observed_flat.max())
            cpick = mpl.cm.ScalarMappable(norm=cnorm,cmap=cmbr)
            cpick.set_array([])
            if calo_observed_flat.max() - calo_observed_flat.min() > 0:
                ix,iy,iz = np.mgrid[-3:3:35j,-3:3:35j,4:15:20j]
                calo_observed_flat_normalized = np.clip((calo_observed_flat - calo_observed_flat.min())/(calo_observed_flat.max() - calo_observed_flat.min()), 0, 1)
                colors = [[x,0,1-x,x] for x in calo_observed_flat_normalized]
                ax1.scatter(ix, iy, iz, s=100, c=colors, marker='o', edgecolor='none')
                plt.colorbar(cpick, ax=ax1, fraction=0.02)

            calo_observed_likelihood_flat = infer_observation_importance * ((np.log(calo_simulated_flat) * calo_observed_flat) - calo_simulated_flat - to_numpy(to_variable(calo_observed_flat + 1).lgamma()))
            ax11.title.set_text('Log-likelihood of observed calorimeter')
            ax11.set_xlabel('x')
            ax11.set_ylabel('y')
            ax11.set_zlabel('z')
            ax11.set_aspect('equal')
            cmbr = mpl.colors.LinearSegmentedColormap.from_list('blue_to_red',['b','r'])
            cnorm = mpl.colors.Normalize(vmin=calo_observed_likelihood_flat.min(),vmax=calo_observed_likelihood_flat.max())
            cpick = mpl.cm.ScalarMappable(norm=cnorm,cmap=cmbr)
            cpick.set_array([])
            if calo_observed_likelihood_flat.max() - calo_observed_likelihood_flat.min() > 0:
                ix,iy,iz = np.mgrid[-3:3:35j,-3:3:35j,4:15:20j]
                calo_observed_likelihood_flat_normalized = np.clip((calo_observed_likelihood_flat - calo_observed_likelihood_flat.min())/(calo_observed_likelihood_flat.max() - calo_observed_likelihood_flat.min()), 0, 1)
                colors = [[x,0,1-x,1-x] for x in calo_observed_likelihood_flat_normalized]
                ax11.scatter(ix, iy, iz, s=100, c=colors, marker='o', edgecolor='none')
                plt.colorbar(cpick, ax=ax11, fraction=0.02)

            ax2.axvline(float(ground_truth_trace.result[3]), color='gray', linestyle='dashed', linewidth=2)
            ax3.axvline(float(ground_truth_trace.result[0]), color='gray', linestyle='dashed', linewidth=2)
            ax4.axvline(float(ground_truth_trace.result[1]), color='gray', linestyle='dashed', linewidth=2)
            ax5.axvline(float(ground_truth_trace.result[2]), color='gray', linestyle='dashed', linewidth=2)

            # ax2.text(0.5, 0.4, 'ground_truth={:.3f} (dashed)'.format(float(ground_truth_trace.result[3])), ha='center', va='center', transform=ax2.transAxes)
            ax3.text(0.5, 0.4, 'ground_truth={:.3f} (dashed)'.format(float(ground_truth_trace.result[0])), ha='center', va='center', transform=ax3.transAxes)
            ax4.text(0.5, 0.4, 'ground_truth={:.3f} (dashed)'.format(float(ground_truth_trace.result[1])), ha='center', va='center', transform=ax4.transAxes)
            ax5.text(0.5, 0.4, 'ground_truth={:.3f} (dashed)'.format(float(ground_truth_trace.result[2])), ha='center', va='center', transform=ax5.transAxes)

            ground_truth_final_state_momenta = ground_truth_trace.result[4:4+(30*8)]
            ground_truth_final_state_momenta = ground_truth_final_state_momenta[ground_truth_final_state_momenta > -99999].view(-1,8)
            for i in range(ground_truth_final_state_momenta.size(0)):
                if float(ground_truth_final_state_momenta[i, 0]) > -99999:
                    ax8.axvline(float(ground_truth_final_state_momenta[i, 0]), color='gray', linestyle='dashed', linewidth=2)
                if float(ground_truth_final_state_momenta[i, 1]) > -99999:
                    ax9.axvline(float(ground_truth_final_state_momenta[i, 1]), color='gray', linestyle='dashed', linewidth=2)
                if float(ground_truth_final_state_momenta[i, 2]) > -99999:
                    ax10.axvline(float(ground_truth_final_state_momenta[i, 2]), color='gray', linestyle='dashed', linewidth=2)
                if float(ground_truth_final_state_momenta[i, 3]) > -99999:
                    ax13.axvline(float(ground_truth_final_state_momenta[i, 3]), color='gray', linestyle='dashed', linewidth=2)
                if float(ground_truth_final_state_momenta[i, 4]) > -99999:
                    ax14.axvline(float(ground_truth_final_state_momenta[i, 4]), color='gray', linestyle='dashed', linewidth=2)
                if float(ground_truth_final_state_momenta[i, 5]) > -99999:
                    ax15.axvline(float(ground_truth_final_state_momenta[i, 5]), color='gray', linestyle='dashed', linewidth=2)

        # plt.subplots_adjust(wspace=0.25)
        if file_name is not None:
            plt.savefig(file_name, bbox_inches='tight')


def plot_trace(trace, file_name=None):
    data = trace.samples_observed[0].distribution.mean.view(35,35,20).data.numpy()
    data_flat = data.reshape(-1)
    data_flat_min = min(data_flat) * min_energy_deposit
    data_flat_max = max(data_flat) * min_energy_deposit
    data_flat_total = np.sum(data_flat) * min_energy_deposit

    particles = trace.result[4:4+(30*8)]
    particles = particles[particles > -99999].view(-1,8)

    trace_text = [str(trace),
                  '\nlatents',
                  'p_x      : {}'.format(float(trace.result[0])),
                  'p_y      : {}'.format(float(trace.result[1])),
                  'p_z      : {}'.format(float(trace.result[2])),
                  'channel  : {}'.format(int(trace.result[3])),
                  'particles: {}'.format(particles.size(0)),
                  str(particles),
                  'calorimeter',
                  'min   : {:.3f} GeV'.format(data_flat_min),
                  'max   : {:.3f} GeV'.format(data_flat_max),
                  'total : {:.3f} GeV'.format(data_flat_total)]
    trace_text = '\n'.join(trace_text)
#     print(trace_text)

    fig = plt.figure(figsize=(15, 4))
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')

    ax1.text(0, 0, trace_text)
    ax1.axis('off')
    cmbr = mpl.colors.LinearSegmentedColormap.from_list('blue_to_red',['b','r'])
    cnorm = mpl.colors.Normalize(vmin=data_flat_min,vmax=data_flat_max)
    cpick = mpl.cm.ScalarMappable(norm=cnorm,cmap=cmbr)
    cpick.set_array([])

    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    ax2.set_zlabel('z')
    ax2.set_aspect('equal')

    if data_flat.max() - data_flat.min() > 0:
        ix,iy,iz = np.mgrid[-3:3:35j,-3:3:35j,4:15:20j]
        data_flat_normalized = np.clip((data_flat - data_flat.min())/(data_flat.max() - data_flat.min()), 0, 1)
        colors = [[x,0,1-x,x] for x in data_flat_normalized]
        ax2.scatter(ix, iy, iz, s=100, c=colors, marker='o', edgecolor='none')

        plt.colorbar(cpick,label='Deposited energy (GeV)',fraction=0.02)

    if file_name is not None:
        plt.savefig(file_name, bbox_inches='tight')

## Inspect the prior
We construct an empirical distribution of the prior, using a given number of traces. Beware, large number of traces can take a long time. traces=500 gives a reasonable-looking plot.


In [ ]:
model = pyprob.RemoteModel('tcp://localhost:2345') # replace accordingly
prior_dist = model.prior_distribution(num_traces=5)

Separate (marginalize) the distribution into distributions over momenta and channel, by mapping a function. (Physics)

In [ ]:

prior_dist_px = prior_dist.map(lambda x: float(x[0]))
prior_dist_py = prior_dist.map(lambda x: float(x[1]))
prior_dist_pz = prior_dist.map(lambda x: float(x[2]))
prior_dist_channel = prior_dist.map(lambda x: int(x[3]))


We can compute expectations under any distribution (prior or posterior):

In [ ]:

prior_dist_pz.expectation(lambda x: math.sin(x)/x)


We can also sample from these distributions, and inspect values and weights.

In [ ]:

prior_dist_pz.sample()
prior_dist_channel.values
prior_dist_channel.weights

We can plot the prior distribution.

In [ ]:
plot_distribution(prior_dist, title='Prior distributions')

We can execute the model, inspect resulting traces and plot them.

In [ ]:
trace = next(model._trace_generator())
plot_trace(trace)

## Inference


In [ ]:

def infer(ground_truth_trace, num_traces=10, inference_engine=InferenceEngine.IMPORTANCE_SAMPLING):
    observation = ground_truth_trace.samples_observed[0].distribution.mean * min_energy_deposit
    posterior = model.posterior_distribution(num_traces, inference_engine, observation=observation)
    return posterior


Inference with Metropolis Hastings

We run inference with single-site Metropolis Hastings.

First we generate a trace to give us ground truth latents and corresponding calorimeter output

In [ ]:
ground_truth_trace = next(model._trace_generator())
plot_trace(ground_truth_trace)

We run inference in the model, conditioning on the calorimeter output from ground_truth_trace.

In [ ]:
posterior_metropolis_hastings = infer(ground_truth_trace,100, inference_engine=InferenceEngine.LIGHTWEIGHT_METROPOLIS_HASTINGS)

In [ ]:
plot_distribution(posterior_metropolis_hastings, title=posterior_metropolis_hastings.name, ground_truth_trace=ground_truth_trace)

## Inference Compilation

In [ ]:
#Train inference network

# pyprob.set_cuda(True)
# model._inference_network = None
model.learn_inference_network(num_traces=20, observe_embedding=pyprob.nn.ObserveEmbedding.CONVNET_3D_4C, observe_reshape=[1,35,35,20], batch_size=4, valid_size=4)


Run inference using proposals from the trained inference network

In [ ]:
posterior_inference_compilation = infer(ground_truth_trace, 10, inference_engine=InferenceEngine.IMPORTANCE_SAMPLING_WITH_INFERENCE_NETWORK)

In [ ]:

plot_distribution(posterior_inference_compilation, title=posterior_inference_compilation.name, ground_truth_trace=ground_truth_trace)

In [ ]:
:
plot_distribution(posterior_inference_compilation.unweighted(), title=posterior_inference_compilation.name, ground_truth_trace=ground_truth_trace)